In [1]:
#default_exp modules.head

# SSD Head for object detection
> As proposed in the paper, we use Single Shot Detector (SSD) for object detection. The header predicts the
> offsets to different, predefined anchors. For each anchor we predict the offsets x, y, z, w, l, h and angle.
> We also receive the match certainty for each anchor.


## 00 - Prerequesits

In [2]:
import sys
#sys.path.append("/home/qhs67/git/bachelorthesis_sven_thaele/code/")
sys.path.insert(0, "../pointpillars")
from utils.io import read_config

### 00.1 - Imports

In [3]:
#export
import logging
import ast
import torch
import torch.nn as nn
import torch.nn.functional as F

logger = logging.getLogger(__name__)

## 01 - Head module

In [4]:
#export
class Head(nn.Module):

    def __init__(self, head_cfg):
        """Definition of the relevant module attributes and conv layers"""

        logger.info("Initialize detection head...")
        super(Head, self).__init__()
        # TODO: define in channel properly
        self.num_anchors = len(ast.literal_eval(head_cfg['anchors']))
        # x,y,z,h,w,l,theta (maybe occupancy to define if object is present)
        self.num_attributes = 7

        self.conv_cls = nn.Conv2d(384, self.num_anchors, 1)
        self.conv_occ = nn.Conv2d(384, self.num_anchors, 1)
        self.conv_head = nn.Conv2d(384, self.num_anchors, 1)
        #self.conv_occ = nn.Conv2d(384, self.num_anchors * 2, 1)
        #self.conv_head = nn.Conv2d(384, self.num_anchors * 2, 1)

        self.conv_boxes = nn.Conv2d(384, self.num_anchors * self.num_attributes, 1)

        logger.debug("Detection head init complete.")


    def forward(self, batch: torch.Tensor) -> tuple:
        """
        :param batch: a batch of data coming from the feature net

        :returns:   (pred_occ(N, H, W, nb_anchors, 2),
                     pred_cls(N, H, W, nb_anchors),
                     pred_head(N, H, W, nb_anchors, 2),
                     pred_boxes(N, H, W, nb_anchors, nb_attributes=7))

                     Meaning of the values:
                     pred_occ gives the confidence of an object occupying this anchor
                     pred_cls gives the predicted class of the object in this anchor
                     pred_head gives the binary prediction of the direction the object is facing (away or towards cam)
                     pred_box gives the dimension deltas to this anchors center (also dependant on pillar center)
        """
        logger.info("Forward through detection head...")

        pred_cls = self.conv_cls(batch)
        pred_occ = self.conv_occ(batch)
        pred_head = self.conv_head(batch)
        pred_boxes = self.conv_boxes(batch)
        C, H, W = pred_boxes.shape[1:]
        # permute (batch_size, n_anchors, H, W) -> (batch_size, H, W, n_anchors)
        pred_cls = pred_cls.permute(0, 2, 3, 1)
        pred_occ = pred_occ.permute(0, 2, 3, 1)
        pred_head = pred_head.permute(0, 2, 3, 1)
        #pred_occ = pred_occ.reshape(-1, self.num_anchors, 2, H, W).permute(0, 3, 4, 1, 2)
        #pred_head = pred_head.reshape(-1, self.num_anchors, 2, H, W).permute(0, 3, 4, 1, 2)
        # resize and permute to fit the correct dimensions
        # out: (batch_size, H, W, n_anchors, 7)
        pred_boxes = pred_boxes.reshape(-1, self.num_anchors, self.num_attributes, H, W).permute(0, 3, 4, 1, 2)



        """
        # h, l, w must be positive even with random weights, use relu to ensure
        pred_boxes_hwl = pred_boxes[:,:,:,:,3:6]
        F.relu(pred_boxes_hwl, inplace=True)
        pred_boxes = torch.cat((pred_boxes[:,:,:,:,:3], pred_boxes_hwl, pred_boxes[:,:,:,:,6:]), dim=4)
        del pred_boxes_hwl
        """

        return pred_occ, pred_cls, pred_head, pred_boxes



In [5]:
torch.cuda.empty_cache()
head_cfg = read_config()['head']

tens = torch.rand(1, 384, 252, 252).cuda()
head = Head(head_cfg)
head.cuda()
occ, cls, head, box = head(tens)

occ, occ.shape, cls, cls.shape, head, head.shape, box, box.shape


KeyboardInterrupt: 

In [ ]:
pred_boxes = torch.rand(4, 250, 250, 5, 7) - 1

tens = pred_boxes[:,:,:,:,4:]

out = F.softplus(tens)

tens, tens.shape, out, out.shape